In [10]:
import pandas as pd
import numpy as np
import os
from random import uniform
from sklearn.datasets import make_classification



In [2]:
os.chdir('../')


In [3]:
from mc_hammer.mchammer import mchammer

In [4]:
def mc_hammer_test(x,null_method,cluster_method,k = None,eps = None,min_samples = None):
    mch = mchammer()
    mch.get_null_distributions(x,null_method,repeats = 100)
    return mch.get_q_scores(cluster_method,k = k,eps=eps,min_samples=min_samples)

In [5]:
def big_mc_hammer(x_list,null_method,cluster_method,k_list = [2,4,5],eps_list = [0.5],min_samples_list = [4,5,7]):
    if cluster_method == 'DBSCAN':
        true_list=[mc_hammer_test(
            l,
            null_method,
            cluster_method,
            k=None,
            eps = j,
            min_samples=i
        ) for l in x_list for i in min_samples_list for j in eps_list]
    else:
        true_list=[mc_hammer_test(
            j,
            null_method,
            cluster_method,
            k = i,
            eps = None,
            min_samples = None
        ) for j in x_list for i in k_list]
    true_dict = {i:[true_list[j][i] for j in range(len(true_list))] for i in true_list[0].keys()}
    true_dict_res = {k:v.count(True)/len(v)*100 for k,v in true_dict.items()}
    return true_dict_res

In [ ]:
def test_blobs(k,noise,seperation,test_true):
    print(str(k) + str(noise) + str(seperation))
    x_list =[make_classification(
        n_samples = 100,
        n_features = 10,
        n_informative = int((10-10*noise)),
        n_redundant = int(10*noise),
        n_classes = k,
        n_clusters_per_class = 1,
        class_sep = seperation,
        random_state = i
    )[0] for i in range(100)]
    null_method_list = ['pca_trans','random_order','min_max']
    cluster_method_list = ['K_Means']
    
    if test_true:
        k_range = [k]
    else:
        k_range = [2,4,5].remove(k)
    
    
    results_dict = {i +'_' + j:big_mc_hammer(
        x_list = x_list,
        null_method = i,
        cluster_method = j,
        k_list = k_range
    ) for i in null_method_list for j in cluster_method_list}
    
    results_df = pd.DataFrame(results_dict).T
    return(results_df)

In [ ]:
null_method_list = ['pca_trans','random_order','min_max']
cluster_method_list = ['K_Means','DBSCAN','spectral_clustering']
x_list = [np.random.rand(100,3) for i in range(100)]
null_results_dict = {i +'_' + j:big_mc_hammer(
    x_list = x_list,
    null_method = i,
    cluster_method = j
) for i in null_method_list for j in cluster_method_list}

In [ ]:
null_results_dict

In [ ]:
pd.DataFrame(null_results_dict).T

In [ ]:
k_list = [2,4,5]
noise_list = [0.1,0.3,0.5]
sep_list = [0.1,0.5,1]
blob_test = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = True
) for i in k_list for j in noise_list for l in sep_list}

In [ ]:
blob_test

In [ ]:
blob_test_false = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = False
) for i in k_list for j in noise_list for l in sep_list}

In [6]:
null_method_list = ['pca_trans','random_order','min_max']
cluster_method_list = ['K_Means','DBSCAN','spectral_clustering']
x_list = [np.random.rand(100,3) for i in range(100)]
null_results_dict = {i +'_' + j:big_mc_hammer(
    x_list = x_list,
    null_method = i,
    cluster_method = j
) for i in null_method_list for j in cluster_method_list}

In [7]:
null_results_dict

{'pca_trans_K_Means': {'huberts_gamma': 0.0,
  'norm_gamma': 97.66666666666667,
  'sillhouette_euclidean': 55.00000000000001,
  'sillhouette_cosine': 3.0,
  'CH': 55.333333333333336,
  'DB': 39.666666666666664},
 'pca_trans_DBSCAN': {'huberts_gamma': 0.0,
  'norm_gamma': 0.0,
  'sillhouette_euclidean': 0.0,
  'sillhouette_cosine': 0.0,
  'CH': 0.0,
  'DB': 0.0},
 'pca_trans_spectral_clustering': {'huberts_gamma': 2.0,
  'norm_gamma': 88.0,
  'sillhouette_euclidean': 48.0,
  'sillhouette_cosine': 8.333333333333332,
  'CH': 55.00000000000001,
  'DB': 33.33333333333333},
 'random_order_K_Means': {'huberts_gamma': 4.333333333333334,
  'norm_gamma': 4.0,
  'sillhouette_euclidean': 5.666666666666666,
  'sillhouette_cosine': 3.6666666666666665,
  'CH': 3.6666666666666665,
  'DB': 4.0},
 'random_order_DBSCAN': {'huberts_gamma': 0.0,
  'norm_gamma': 0.0,
  'sillhouette_euclidean': 0.0,
  'sillhouette_cosine': 0.0,
  'CH': 0.0,
  'DB': 0.0},
 'random_order_spectral_clustering': {'huberts_gamma':

In [9]:
pd.DataFrame(null_results_dict).T

,huberts_gamma,norm_gamma,sillhouette_euclidean,sillhouette_cosine,CH,DB
pca_trans_K_Means,0.000000,97.666667,55.000000,3.000000,55.333333,39.666667
pca_trans_DBSCAN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
pca_trans_spectral_clustering,2.000000,88.000000,48.000000,8.333333,55.000000,33.333333
random_order_K_Means,4.333333,4.000000,5.666667,3.666667,3.666667,4.000000
random_order_DBSCAN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
random_order_spectral_clustering,5.666667,3.666667,6.333333,6.333333,3.333333,5.000000
min_max_K_Means,15.666667,3.333333,5.666667,6.333333,3.333333,5.666667
min_max_DBSCAN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
min_max_spectral_clustering,16.000000,2.666667,6.333333,5.000000,4.000000,4.666667


In [28]:
k_list = [2,4,5]
noise_list = [0.1,0.3,0.5]
sep_list = [0.1,0.5,1]
blob_test = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = True
) for i in k_list for j in noise_list for l in sep_list}

20.10.1
20.10.5
20.11
20.30.1
20.30.5
20.31
20.50.1
20.50.5
20.51
40.10.1
40.10.5
40.11
40.30.1
40.30.5
40.31
40.50.1
40.50.5
40.51
50.10.1
50.10.5
50.11
50.30.1
50.30.5
50.31
50.50.1
50.50.5
50.51


In [29]:
blob_test

{'k-2_noise-0.1_sep-0.1':                       huberts_gamma  norm_gamma  sillhouette_euclidean  \
 pca_trans_K_Means               0.0         2.0                    3.0   
 random_order_K_Means           53.0         2.0                  100.0   
 min_max_K_Means                 0.0         6.0                   63.0   
 
                       sillhouette_cosine     CH     DB  
 pca_trans_K_Means                    0.0    0.0    2.0  
 random_order_K_Means                98.0  100.0  100.0  
 min_max_K_Means                     62.0   65.0   76.0  ,
 'k-2_noise-0.1_sep-0.5':                       huberts_gamma  norm_gamma  sillhouette_euclidean  \
 pca_trans_K_Means               0.0         2.0                    2.0   
 random_order_K_Means           53.0         2.0                  100.0   
 min_max_K_Means                 0.0         7.0                   67.0   
 
                       sillhouette_cosine     CH     DB  
 pca_trans_K_Means                    0.0    0.0    1.0

In [30]:
blob_test_false = {'k-' + str(i) +'_noise-' +str(j) + '_sep-' + str(l):test_blobs(
    k = i,
    noise = j,
    seperation = l,
    test_true = False
) for i in k_list for j in noise_list for l in sep_list}

20.10.1


TypeError: 'NoneType' object is not iterable